In [23]:
import pandas as pd
import os
import tempfile
from sqlalchemy import create_engine
from dotenv import load_dotenv
import re

### Extraccion

In [24]:

# Obtener la ruta de la carpeta actual y subir una carpeta
parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ruta de descarga personalizada
Data_path = os.path.join(parent_directory, "Data")
productos_path = os.path.join(Data_path, "productos_prueba_v2.txt")

In [25]:
tickets_path = os.path.join(Data_path, "tickets_prueba_v2.txt")
df_tickets = pd.read_csv(tickets_path, sep=';', encoding='utf-8', index_col=None)

C:\Users\Joa\AppData\Local\Temp\ipykernel_25384\562491179.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tickets = pd.read_csv(tickets_path, sep=';', encoding='utf-8', index_col=None)


### Transformacion

In [26]:

with open(productos_path, 'r', encoding='utf-8',errors="replace") as file:
    content = file.read()
# Eliminar todas las apariciones de '_x000D_'
content = content.replace('_x000D_', '')
# Reemplazar 'RF.I;' por 'RF.I:'
content = content.replace('RF.I;', 'RF.I:')

# Crear archivo temporal para los datos procesados
with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as temp_file:
    temp_file.write(content)
    temp_file_path = temp_file.name  # Obtener la ruta del archivo temporal

# Procesar las líneas del archivo temporal
with open(temp_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

new_lines = []
buffer_line = ""

for line in lines:
    line = line.strip()

    # Si la línea empieza con un número, es una nueva línea de datos
    if re.match(r"^\d+", line):
        if buffer_line:
            new_lines.append(buffer_line)  # Guardamos la línea acumulada
        buffer_line = line  # Iniciar una nueva línea
    else:
        buffer_line += " " + line  # Concatenamos con la anterior

# Agregar la última línea acumulada si existe
if buffer_line:
    new_lines.append(buffer_line)

# Guardar el contenido corregido en el archivo temporal
with open(temp_file_path, 'w', encoding='utf-8') as file:
    file.write('\n'.join(new_lines))
# Flag para verificar si hubo cambios en las líneas
changes_made = True
while changes_made:    
    changes_made = False
    # Procesar las líneas del archivo temporal
    with open(temp_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    new_lines.append(lines[0])

    for line in lines[1:]:
        parts = line.strip().split(';')

        if len(parts) >= 4 and not parts[3].isdigit():
            parts[2] = parts[2] + ' ' + parts[3]
            parts = parts[:3] + parts[4:]

            # Si hubo un cambio, marcamos la bandera como True
            changes_made = True


        new_line = ';'.join(parts)
        new_lines.append(new_line)

    # Guardar el contenido corregido en el archivo temporal
    with open(temp_file_path, 'w', encoding='utf-8') as file:
        file.write('\n'.join(new_lines))




In [27]:
# Leer el archivo procesado con pandas
df = pd.read_csv(temp_file_path, sep=';', encoding='utf-8')

df.columns = df.columns.str.strip()  # Elimina espacios al inicio y al final

### Carga

In [29]:
# 1. Tabla Sectores (eliminar sector con valor 0)
df_sectores = df[['id_sector', 'sector','idcadena']].drop_duplicates().reset_index(drop=True)
df_sectores = df_sectores[df_sectores['sector'] != 'SECTOR_PRUEBA']

# 2. Tabla Secciones (eliminar secciones con id 0 y 1)
df_secciones = df[['id_seccion', 'seccion','idcadena']].drop_duplicates().reset_index(drop=True)

# 3. Tabla Categorias (eliminar categoria con id 1)
df_categorias = df[['id_categoria', 'categoria','idcadena']].drop_duplicates().reset_index(drop=True)
df_categorias = df_categorias[df_categorias['categoria'] != 'CATEGORIA_PRUEBA']

# 4. Tabla Subcategorias (eliminar subcategoria con id 1)
df_subcategorias = df[['id_subcategoria', 'subcategoria','idcadena']].drop_duplicates().reset_index(drop=True)
df_subcategorias = df_subcategorias[df_subcategorias['subcategoria'] != 'SUBCAPRUEBA']

# 8. Tabla Productos (principal)
df_productos = df[['idcadena','eancode', 'descripcion', 'id_sector', 'id_seccion', 'id_categoria', 'id_subcategoria', 
                   'fabricante', 'marca', 'contenido', 'pesovolumen', 'unidadmedida', 'ultmodificacion', 'id','granfamilia', 'familia']]


# Eliminar las filas donde id_seccion sea 61
df_productos = df_productos[df_productos['id_seccion'] != 61]
df_productos = df_productos.drop_duplicates(subset=['id_sector', 'id_seccion', 'eancode', 'id_categoria', 'fabricante'], keep='first')


In [ ]:

# Cargar las variables de entorno desde .env
load_dotenv()

DB_USER = os.getenv("DB_USER")  # Puedes agregar esto al .env si prefieres
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = "spaceman"

# Crear la URL de conexión a PostgreSQL
database_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Crear una conexión a la base de datos usando SQLAlchemy
engine = create_engine(database_url)

In [ ]:

df_categorias.to_sql('categorias', con=engine, schema='maestros', if_exists='append', index=False)


df_sectores.to_sql('sectores', con=engine, schema='maestros', if_exists='append', index=False)

df_secciones.to_sql('secciones', con=engine, schema='maestros', if_exists='append', index=False)

df_subcategorias.to_sql('subcategorias', con=engine, schema='maestros', if_exists='append', index=False)

df_productos.to_sql('productos', con=engine, schema='maestros', if_exists='append', index=False)

df_tickets.to_sql('tickets', con=engine, schema='maestros', if_exists='append', index=False)

66